In [2]:
# import random as rnd
# import secrets as scr

from datetime import datetime
from random import SystemRandom

rnd = SystemRandom()


class Lotteria:
    def __init__(self,
                 max_numbers=90,
                 max_extra=90,
                 numbers=6,
                 extra=1):
        self.max_numbers = max_numbers
        self.max_extra = max_extra
        self.numbers = numbers
        self.extra = extra

    @staticmethod
    def pallottoliere(max_nums):
        while True:
            yield rnd.randint(1, max_nums)

    @staticmethod
    def choice(nums, max_nums):
        whole_numbers = list(range(1, max_nums+1))

        def extraction():
            rnd.shuffle(whole_numbers)
            single = rnd.choice(whole_numbers)
            whole_numbers.remove(single)

            return single

        return frozenset([extraction() for _ in range(nums)])

    @staticmethod
    def sample(nums, max_nums):
        whole_numbers = frozenset((range(1, max_nums+1)))
        whole_numbers = sorted(whole_numbers)

        return frozenset(rnd.sample(whole_numbers, k=nums))

    def int(self, nums, max_nums):
        combo = set()
        while len(combo) < nums:
            combo.add(next(self.pallottoliere(max_nums)))

        return frozenset(combo)

    def manySamples(self, backend):
        def sampler(nums, max_nums):
            samples = [backend(nums, max_nums) for _ in range(self._many)]
            rnd.shuffle(samples)

            return rnd.choice(samples)
        return sampler

    @property
    def backend(self):
        return self._backend

    @property
    def many(self):
        return self._many

    def __call__(self, backend='sample', many=0, verbose=True):
        self._backend = backend
        self._many = many

        backend = eval(backend,
                       {'__builtins__': {}},
                       {'choice': self.choice, 'randint': self.int, 'sample': self.sample})

        backend = self.manySamples(backend) if self.many else backend

        combo = backend(self.numbers, self.max_numbers)
        if self.extra:
            extra = backend(self.extra, self.max_extra)

        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        if verbose:
            print(f'Estrazione del: {now}', '\nNumeri Estratti:', *sorted(combo))
            if self.extra:
                print('Superstar:', *sorted(extra))  # type: ignore

        if self.extra:
            return combo, extra # type: ignore
        else:
            return combo

In [3]:
superenalotto = Lotteria(max_numbers=90, max_extra=90, numbers=6, extra=0)

In [4]:
superenalotto(backend='sample', many=1_000_000);

Estrazione del: 10/06/2023 09:57 
Numeri Estratti: 1 8 15 29 59 82


In [5]:
eurojack = Lotteria(max_numbers=50, max_extra=12, numbers=5, extra=2)

In [6]:
eurojack(backend='sample', many=500_000);

Estrazione del: 10/06/2023 09:57 
Numeri Estratti: 5 11 34 44 46
Superstar: 4 9


In [7]:
eurojack(backend='randint', many=500_000);

Estrazione del: 10/06/2023 09:58 
Numeri Estratti: 1 15 27 43 48
Superstar: 1 2


In [8]:
# for b in ['choice', 'int', 'sample']:
#    print('######')
#    eurojack(backend=b, many=0)
#    print('backend:', eurojack.backend)
#
# print('\n################# many ####################\n')
#
# for b in ['choice', 'int', 'sample']:
#    print('######')
#    eurojack(backend=b)
#    print('backend:', eurojack.backend)

In [9]:
superenalotto.extra = 1
superenalotto();

Estrazione del: 10/06/2023 09:58 
Numeri Estratti: 1 2 9 30 71 81
Superstar: 22


In [10]:
superenalotto(backend='randint');

Estrazione del: 10/06/2023 09:58 
Numeri Estratti: 1 8 28 33 35 83
Superstar: 50


In [11]:
superenalotto.extra

1

In [12]:
superenalotto.many

0

In [15]:
winForLife = Lotteria(20, 20, 10, 1)
winForLife(backend='choice', many=100_000);

Estrazione del: 10/06/2023 09:59 
Numeri Estratti: 1 6 8 9 12 14 15 17 19 20
Superstar: 14


In [14]:
# trying to set backend via attribute: fails as expected
winForLife.backend = 'int'

AttributeError: property 'backend' of 'Lotteria' object has no setter